In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/1n/nvwgxs_54r55k5r7_5zcqtdr0000gn/T/ipykernel_46198/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [9]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv('/Users/paolarivera/Documents/Ironhack/Week 7/Day 1/lab-natural-language-processing-main/data/kg_train.csv',encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [37]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [47]:
from sklearn.model_selection import train_test_split

# Read the dataset
data = pd.read_csv('/Users/paolarivera/Documents/Ironhack/Week 7/Day 1/lab-natural-language-processing-main/data/kg_train.csv', encoding='latin-1')

# Reduce the training set to speed up development
data = data.head(1000)
print(data.shape)
data.fillna("", inplace=True)

# Split the data into train and test sets
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

(1000, 2)


## Data Preprocessing

In [50]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [53]:
import re

def clean_html(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    return text

# Apply the clean_html function
data_train['cleaned_text'] = data_train['text'].apply(clean_html)
data_val['cleaned_text'] = data_val['text'].apply(clean_html)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [55]:
def normalize_text(text):
    # Remove special characters and digits, and keep only words
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove single characters and whitespace
    text = re.sub(r'\b\w\b', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply normalization
data_train['preprocessed_text'] = data_train['cleaned_text'].apply(normalize_text)
data_val['preprocessed_text'] = data_val['cleaned_text'].apply(normalize_text)

## Now let's work on removing stopwords
Remove the stopwords.

In [57]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    return ' '.join([word for word in words if word not in stop_words])

data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [59]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    return ' '.join([lemmatizer.lemmatize(word) for word in words])

data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(lemmatize_text)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lemmatize_text)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

X_train_count = count_vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_count = count_vectorizer.transform(data_val['preprocessed_text'])

print(X_train_count.shape)

(800, 16932)


## Extra features

In [63]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,cleaned_text,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",regard mr nelson smithkindly reply private ema...,1,0,75
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...,able reach oscar supposed send pdb receive,1,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...,huma abedin bim checking pat work jack jake re...,1,0,79
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...,announced monday cant today,1,0,27
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,bank africaagence san pedro bp san pedro cote ...,1,1,1067


## How would work the Bag of Words with Count Vectorizer concept?

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [65]:
tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_tfidf = tfidf_vectorizer.transform(data_val['preprocessed_text'])

print(X_train_tfidf.shape)

(800, 16932)


## And the Train a Classifier?

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, data_train['label'])

# Make predictions
predictions = model.predict(X_val_tfidf)

# Evaluate the model
accuracy = accuracy_score(data_val['label'], predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 0.96


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [76]:
import pandas as pd
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download stopwords if not already available
nltk.download('stopwords')
nltk.download('wordnet')

# Load dataset
data = pd.read_csv('/Users/paolarivera/Documents/Ironhack/Week 7/Day 1/lab-natural-language-processing-main/data/kg_test.csv')  # Change path to your dataset
data.fillna("", inplace=True)  # Handle missing values

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    
    # Apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return " ".join(words)

# Apply cleaning function to text column
data['clean_text'] = data['text'].apply(clean_text)

# Check first few cleaned entries
print(data[['text', 'clean_text']].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/paolarivera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/paolarivera/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/var/folders/1n/nvwgxs_54r55k5r7_5zcqtdr0000gn/T/ipykernel_46198/3564454565.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
/var/folders/1n/nvwgxs_54r55k5r7_5zcqtdr0000gn/T/ipykernel_46198/3564454565.py:23: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


                                                text  \
0  usiness is for the fact that the deceased man ...   
1  They are happy to adjust to the afternoon. I a...   
2  Lael Brainard was confirmed 78-19 this afterno...   
3  H <hrod17@clintonemail.com>Friday March 26 201...   
4  n;"> Dear Good Friend,<br><br><br>I am happy t...   

                                          clean_text  
0  usiness fact deceased man foreigner authorized...  
1  happy adjust afternoon going suggest pm start ...  
2  lael brainard confirmed afternoonmiguel e rodr...  
3    h friday march amsbwhoeop rei extended congrats  
4  n dear good friendi happy inform successin get...  


In [92]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert text to Bag of Words representation
count_vectorizer = CountVectorizer(ngram_range=(1,2), max_features=5000)  # Unigrams + Bigrams
X_bow = count_vectorizer.fit_transform(data['clean_text'])

print("BoW feature shape:", X_bow.shape)

BoW feature shape: (5964, 5000)


In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF representation
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(data['clean_text'])

print("TF-IDF feature shape:", X_tfidf.shape)

TF-IDF feature shape: (5964, 5000)


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# Encode labels: Assume 'spam' = 1, 'ham' = 0
data['label'] = data['text'].apply(lambda x: 1 if "spam" in x.lower() else 0)

# Split into training and testing sets
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, data['label'], test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(X_tfidf, data['label'], test_size=0.2, random_state=42)

# Train MultinomialNB with BoW
nb_bow = MultinomialNB()
nb_bow.fit(X_train_bow, y_train)
y_pred_bow = nb_bow.predict(X_test_bow)

# Train MultinomialNB with TF-IDF
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)

# Evaluate using F1 Score
f1_bow = f1_score(y_test, y_pred_bow)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

print("F1 Score (BoW):", f1_bow)
print("F1 Score (TF-IDF):", f1_tfidf)

F1 Score (BoW): 0.13636363636363635
F1 Score (TF-IDF): 0.0


In [98]:
# Check class distribution
print(data['label'].value_counts())

# Undersample HAM to balance SPAM
min_class_count = data['label'].value_counts().min()
data_balanced = data.groupby('label').apply(lambda x: x.sample(min_class_count)).reset_index(drop=True)


label
0    5924
1      40
Name: count, dtype: int64


/var/folders/1n/nvwgxs_54r55k5r7_5zcqtdr0000gn/T/ipykernel_46198/951715527.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_balanced = data.groupby('label').apply(lambda x: x.sample(min_class_count)).reset_index(drop=True)


In [102]:
# Try limiting features
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=1000, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(data_balanced['clean_text'])

count_vectorizer = CountVectorizer(ngram_range=(1,1), max_features=1000, stop_words='english')
X_bow = count_vectorizer.fit_transform(data_balanced['clean_text'])

In [104]:
# Add text length and uppercase letter count
data_balanced['text_len'] = data_balanced['clean_text'].apply(len)
data_balanced['uppercase_count'] = data_balanced['clean_text'].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add suspicious words
suspicious_words = ["free", "win", "money", "prize", "cash", "lottery", "offer"]
data_balanced['suspicious_count'] = data_balanced['clean_text'].apply(lambda x: sum(word in x for word in suspicious_words))

# Merge features into dataset
from scipy.sparse import hstack
X_bow_extended = hstack((X_bow, data_balanced[['text_len', 'uppercase_count', 'suspicious_count']].values))
X_tfidf_extended = hstack((X_tfidf, data_balanced[['text_len', 'uppercase_count', 'suspicious_count']].values))

In [106]:
# Split again
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_extended, data_balanced['label'], test_size=0.2, random_state=42)

# Train the MultinomialNB model
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

# Evaluate
print("F1 Score:", f1_score(y_test, y_pred))

F1 Score: 0.5


In [108]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=500, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(data_balanced['clean_text'])

In [110]:
nb = MultinomialNB(alpha=0.1)
nb.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [112]:
data_balanced['punctuation_count'] = data_balanced['clean_text'].apply(lambda x: sum(1 for c in x if c in "!$%&?"))
data_balanced['uppercase_ratio'] = data_balanced['clean_text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

In [114]:
import numpy as np
data_balanced['log_text_len'] = np.log1p(data_balanced['text_len'])

In [118]:
# Drop NaN values and keep only relevant columns
data = data[['text', 'label']].dropna()
data.fillna("", inplace=True)

# Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply text cleaning
data["clean_text"] = data["text"].apply(preprocess_text)

# Feature Engineering
data['punctuation_count'] = data['text'].apply(lambda x: sum(1 for c in x if c in "!$%&?"))
data['uppercase_ratio'] = data['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
data['text_len'] = data['text'].apply(len)
data['log_text_len'] = np.log1p(data['text_len'])  # Log transform for better distribution

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["label", "text"]), data["label"], test_size=0.2, random_state=42)

# TF-IDF Vectorization with Bi-Grams
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=500, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train["clean_text"])
X_test_tfidf = tfidf_vectorizer.transform(X_test["clean_text"])

# Convert additional features to numpy arrays and concatenate with TF-IDF features
X_train_features = np.hstack([
    X_train_tfidf.toarray(),
    X_train[['punctuation_count', 'uppercase_ratio', 'log_text_len']].values
])
X_test_features = np.hstack([
    X_test_tfidf.toarray(),
    X_test[['punctuation_count', 'uppercase_ratio', 'log_text_len']].values
])

# Train MultinomialNB classifier with alpha=0.1
nb = MultinomialNB(alpha=0.1)
nb.fit(X_train_features, y_train)

# Predictions
y_pred = nb.predict(X_test_features)

# Evaluate F1 Score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

F1 Score: 0.9696376101860921
